<h2 align="center">Automatic Machine Learning with H2O AutoML</h2>

### Importing Packages

In [ ]:
import pandas as pd
pd.options.display.max_rows = 999
import numpy as np
import matplotlib.pyplot as plt

### Loading and Exploring the Data

In [ ]:
xls = pd.ExcelFile(r'bank_term_deposit_marketing_analysis.xlsx')

In [ ]:
xls.sheet_names

In [ ]:
client_info = pd.read_excel(xls, 'CLIENT_INFO')
load_history = pd.read_excel(xls, 'LOAN_HISTORY')
marketing_history = pd.read_excel(xls, 'MARKETING HISTORY')
subscription_history = pd.read_excel(xls, 'SUBSCRIPTION HISTORY')

In [ ]:
client_info.head()

In [ ]:
load_history.head()

In [ ]:
marketing_history.head()

In [ ]:
subscription_history.head()

In [ ]:
df = pd.merge(client_info, load_history, on=['ID'])
df = pd.merge(df, marketing_history, on=['ID'])
df = pd.merge(df, subscription_history, on=['ID'])
df.head()

In [ ]:
df = df.drop('ID', axis=1)

In [ ]:
df.head(3)

### Data Prep & Start H2O

In [ ]:
import h2o
h2o.init(max_mem_size='8G')

In [ ]:
h2o_df = h2o.H2OFrame(df)

In [ ]:
h2o_df.describe()

In [ ]:
train,test = h2o_df.split_frame(ratios=[.75])

x = train.columns
y = 'TERM_DEPOSIT'
x.remove(y)

### Run H2O AutoML

In [ ]:
from h2o.automl import H2OAutoML

In [ ]:
aml = H2OAutoML(max_runtime_secs=600,
                balance_classes=True,
                stopping_metric='logloss',
                project_name='Final',
                seed=1
               )

%time aml.train(x=x, y=y, training_frame=train)

### AutoML Leaderboard and Ensemble Exploration

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [ ]:
se = aml.leader
metalearner = h2o.get_model(se.metalearner()['name'])

In [ ]:
metalearner.varimp()

### Base Learner XGBoost Model Exploration

In [ ]:
model = h2o.get_model('DRF_1_AutoML_1_20230810_175713')

In [ ]:
model.model_performance(test)

In [ ]:
model.varimp_plot(num_of_features=20)